In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

pd.options.mode.chained_assignment = None  # default='warn'
features_in = "data/features.csv"

In [26]:
column_names = [
        'subject','duration','card_class','show_order',
        'fix_freq','sacc_freq','sr_fix_freq','sr_sacc_freq',
        'pupil_diam_right_mean','pupil_diam_right_std','pupil_diam_right_min','pupil_diam_right_max',
        'pupil_diam_left_mean','pupil_diam_left_std','pupil_diam_left_min','pupil_diam_left_max',
        'sr_pupil_diam_right_mean','sr_pupil_diam_right_std','sr_pupil_diam_right_min','sr_pupil_diam_right_max',
        'sr_pupil_diam_left_mean','sr_pupil_diam_left_std','sr_pupil_diam_left_min','sr_pupil_diam_left_max',
        'label'
    ]
    
features = pd.read_csv(features_in, sep='\t')
for col in column_names:
    features[col] = features[col].fillna(0)
    

features.isnull().sum().sum()

0

In [27]:
feature_cols = [
        'fix_freq','sacc_freq','sr_fix_freq','sr_sacc_freq',
        'pupil_diam_right_mean','pupil_diam_right_std','pupil_diam_right_min','pupil_diam_right_max',
        'pupil_diam_left_mean','pupil_diam_left_std','pupil_diam_left_min','pupil_diam_left_max',
        'sr_pupil_diam_right_mean','sr_pupil_diam_right_std','sr_pupil_diam_right_min','sr_pupil_diam_right_max',
        'sr_pupil_diam_left_mean','sr_pupil_diam_left_std','sr_pupil_diam_left_min','sr_pupil_diam_left_max'
    ]
X = features[feature_cols]
Y = features['label']
X_train, X_test, y_train, y_test = \
        train_test_split(X, Y, train_size=0.75,test_size=0.25, random_state=1, shuffle=True) # 70% training and 30% test

In [28]:
# Create Decision Tree classifer object
tree = DecisionTreeClassifier()

# Train Decision Tree Classifer
tree = tree.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = tree.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6538461538461539
